In [1]:
!pip install pandas
!pip install requests
!pip install bs4
!pip install plotly
!pip install numpy
!pip install geopy
!pip install html5lib
!pip install sqlalchemy==1.3.9
!pip install ipython-sql
!pip install psycopg2-binary

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import clear_output
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import html5lib
import psycopg2

In [3]:
html_data = requests.get("https://www.otomoto.pl/oferta/bmw-m5-600km-m5-individual-kolor-carbon-pakiet-ceramiczne-hamulce-salon-pl-ID6DQdad.html")
soup = BeautifulSoup(html_data.content, 'html5lib')

In [4]:
eur = 4.57
print(eur)


4.57


In [5]:
def get_data(link):
    price = soup.find('span',attrs={"class":"offer-price__number"}).text
    price = price.replace(" ","")
    if "PLN" in price:
        price = price.replace("PLN","")
        price = float(price)
    elif "EUR" in price:
        price = price.replace("EUR","")
        price = float(price)
        price = price * eur
    else:
        price = None
    print(price)

In [6]:
price = soup.find('span',attrs={"class":"offer-price__number"}).text
price = price.replace(" ","")
if "PLN" in price:
    price = price.replace("PLN","")
    price = float(price)
elif "EUR" in price:
    price = price.replace("EUR","")
    price = float(price)
    price = price * eur
else:
    price = None
print(price)


649900.0


In [7]:
ID = soup.find('div',attrs={"class":"offer-content__metabar"})
ID = ID.find('div')
ID = ID.find('span',attrs={"id":"ad_id"}).text
print(ID)

6085517757


In [8]:
ID = soup.find('div',attrs={"class":"offer-content__metabar"})

In [9]:
def take_details(soup):
    details = {}
    for info in soup.find_all('li',attrs={"class":"offer-params__item"}):
        key = info.find('span').text.strip()
        value = info.find('div').text.strip()
        details[key] = value
    return details

In [10]:
details = take_details(soup)
details

{'Oferta od': 'Firmy',
 'Kategoria': 'Osobowe',
 'Marka pojazdu': 'BMW',
 'Model pojazdu': 'M5',
 'Rok produkcji': '2021',
 'Przebieg': '10 000 km',
 'Pojemność skokowa': '4 395 cm3',
 'Rodzaj paliwa': 'Benzyna',
 'Moc': '600 KM',
 'Skrzynia biegów': 'Automatyczna',
 'Napęd': '4x4 (dołączany automatycznie)',
 'Typ nadwozia': 'Sedan',
 'Liczba drzwi': '4',
 'Liczba miejsc': '5',
 'Kolor': 'Inny kolor',
 'Faktura VAT': 'Tak',
 'Kraj pochodzenia': 'Polska',
 'Pierwsza rejestracja': '16/05/2021',
 'Pierwszy właściciel': 'Tak',
 'Bezwypadkowy': 'Tak',
 'Serwisowany w ASO': 'Tak',
 'Stan': 'Używane'}

In [11]:
details['Przebieg'] = details['Przebieg'].replace(" km","").replace(" ","")
details['Moc'] = details['Moc'].replace(" KM","")
details['Pojemność skokowa'] = details['Pojemność skokowa'].replace(" cm3","").replace(" ","")


In [12]:
details

{'Oferta od': 'Firmy',
 'Kategoria': 'Osobowe',
 'Marka pojazdu': 'BMW',
 'Model pojazdu': 'M5',
 'Rok produkcji': '2021',
 'Przebieg': '10000',
 'Pojemność skokowa': '4395',
 'Rodzaj paliwa': 'Benzyna',
 'Moc': '600',
 'Skrzynia biegów': 'Automatyczna',
 'Napęd': '4x4 (dołączany automatycznie)',
 'Typ nadwozia': 'Sedan',
 'Liczba drzwi': '4',
 'Liczba miejsc': '5',
 'Kolor': 'Inny kolor',
 'Faktura VAT': 'Tak',
 'Kraj pochodzenia': 'Polska',
 'Pierwsza rejestracja': '16/05/2021',
 'Pierwszy właściciel': 'Tak',
 'Bezwypadkowy': 'Tak',
 'Serwisowany w ASO': 'Tak',
 'Stan': 'Używane'}

In [13]:
# def geturl(url):
#     html_data_url = requests.get(url)
#     soup_url = BeautifulSoup(html_data_url.content, 'html5lib')
#     location = soup.find('a',attrs={"class":"css-1ibwe9h e1nbpvi62"}).text
#     new_row = {}
#     for offer in soup_url.find_all('a',attrs={"class":"offer-title__link"}):
#             new_row = offer['href']

In [14]:
html_data_url = requests.get("https://www.otomoto.pl/osobowe/")
soup_url = BeautifulSoup(html_data_url.content, 'html5lib')

# new_row = {}
# for offer in soup_url.find_all('a',attrs={"class":"offer-title__link"}).find_all('a',attrs={"class":"offer-title__link"}):
#     new_row = get_data(offer['href'])

In [15]:
links = []
new_row = {}
for offer in soup_url.find_all('a',attrs={"class":"offer-title__link"}):
    if "https://www.otomoto.pl" in offer['href']:
        links.append(offer['href'])
print(links)

['https://www.otomoto.pl/oferta/mitsubishi-pajero-mitsubishi-pajero-3-0-v6-gratis-volkswagen-golf-ii-1988-rok-ID6E2xd9.html#7ce9f71ee3', 'https://www.otomoto.pl/oferta/honda-cr-v-bezwypadkowy-pakiet-aero-navi-2xopony-kamera-parktroniki-aso-ID6E2xfr.html#7ce9f71ee3', 'https://www.otomoto.pl/oferta/seat-leon-sprzedam-moje-oczko-w-glowie-ID6E30o2.html#7ce9f71ee3', 'https://www.otomoto.pl/oferta/mercedes-benz-klasa-e-ID6E33Tp.html#7ce9f71ee3', 'https://www.otomoto.pl/oferta/bmw-seria-3-m-pakiet-salon-polska-1-wlasciciel-100-bezwypadkowa-przebieg-19-tys-ID6DYXuL.html#7ce9f71ee3', 'https://www.otomoto.pl/oferta/chevrolet-malibu-z-zalem-sprzedaje-ID6E2xd4.html#7ce9f71ee3', 'https://www.otomoto.pl/oferta/audi-a4-1-8t-quattro-4x4-bfb-163km-serwis-bardzo-zadbana-2-klucze-tempomat-ID6E33Ob.html#7ce9f71ee3', 'https://www.otomoto.pl/oferta/citroen-c4-picasso-2-0-lift-ledy-navigacja-pdc-grey-platinium-ID6DUfzG.html#7ce9f71ee3', 'https://www.otomoto.pl/oferta/chevrolet-captiva-2-2d-184km-4x4-automat-

In [16]:
get_data("https://www.otomoto.pl/oferta/ferrari-f8-tributo-official-ferrari-dealer-ID6E2r6Z.html")

649900.0


In [17]:
print(new_row)

{}


In [ ]:
Link_ERROR = 0
Read_announcements = 0
# page = 1
# while True:
    
for page in range(1,101):  
    try:
        html_data_link = requests.get(f"https://www.otodom.pl/pl/oferty/wynajem/mieszkanie/warszawa?page={page}")
        page += 1
    except Exception:
        print("Link_ERROR")
        continue
    
    soup_link = BeautifulSoup(html_data_link.content, 'html5lib')
    
    if soup_link.find('div',attrs={"class":"css-pi2gf8 e1qwpsp45"}) is None:
        pass
    else:
        break

    for offer in soup_link.find_all('a',attrs={"data-cy":"listing-item-link"}):
            new_row = get_data("https://www.otodom.pl"+ offer['href'])
            
            
            if new_row == None:
                Link_ERROR += 1
            else:
                df_table = df_table.append(new_row, ignore_index=True)
                Read_announcements += 1
                clear_output(wait=True)
                print("Read_announcements ", Read_announcements)
                print("Link_ERROR ",Link_ERROR)     
                
print("Finish")

649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
649900.0
6